# Inspecting Your Data, Part 2: Carpet AMR Data

In this notebook, we show how to investigate Einstein Toolkit data in Python. The Einstein Toolkit uses an AMR library, called Carpet. Carpet is a complicated piece of machinery, so let's see if we can understand some of its moving parts as we go along.

First, let's import the same python tools as we used before:

In [ ]:
%matplotlib inline
import numpy as np
import h5py
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
mpl.rcParams.update({'font.size':18})

Here's the files that Carpet output:

In [ ]:
!ls qc0-mclachlan-cell-t0

Carpet outputs one hdf5 file per "variable group" (or per variable, depending on your settings) per MPI thread. Syou can see that this simulation was run on four MPI threads (though it could be many more openmp threads since Carpet is hybrid-parallelized.) Let's look inside one:

In [ ]:
f = h5py.File('qc0-mclachlan-cell-t0/admbase-lapse.file_0.h5','r')
list(f.keys())

This is the typical format for a Carpet IO file. Let's unpack it:

`ADMBASE::alp`
means that the variable is named `alp` and is defined in the `ADMBASE` thorn. This is the lapse in the ADM formulation of the Einstein equations.

`it=0`
means this output is from the zeroth iteration of the simulation. It's initial data.

`tl=0` refers to the subcycling in time that Carpet can do. (Finer grids can be evolved at shorter timesteps than coarser grids.) Unless you explicitly ask, Carpet will always output data with `tl=0`. 

`rl=0` specifies the coarseness of the grid. The grid spacing $\Delta x$ is divided by a factor of two for each increased refinement level. So `rl=6` has a grid spacing $2^6$ times smaller than `rl=0`. 

`c=0` specifies the *component* of the grid that Carpet has output. When Carpet distributes the grid structure onto a distributed memory supercomputer, it has to give different pieces of the grid to different MPI threads. Each of these pieces of a grid is called a component. Carpet chooses to output components as it likes to balance the load as best it can. In this example, carpet assigned one component at each refinement level to each MPI thread. This is a typical configuration but *it is not guaranteed.*

Let's look in more detail what's inside that `Parameters and Global Attributes` group. A group is a kind of subdirectory in an HDF5 file. It's a way of making the files self-describing. Carpet puts useful metadata in this group.

In [ ]:
list(f['Parameters and Global Attributes'].keys())

You can probably guess what some of these contain. The `Datasets` dataset just contains a list of variable names the file contains. Ours only contains the lapse.

In [ ]:
f['Parameters and Global Attributes/Datasets'].value

The other datasets in this group are more interesting. `All Parameters` contains what is essentially a copy of the parameter file used to run the simulation:

In [ ]:
print(str(f['Parameters and Global Attributes/All Parameters'].value,encoding='utf'))

Finally, the `Grid Structure v5` dataset contains a description of how Carpet structured the AMR hierarchy and broke up the AMR grids into components:

In [ ]:
print(str(f['Parameters and Global Attributes/Grid Structure v5'].value,encoding='utf'))

This isn't meant to be human readable---it's a direct dump of the internal represnetation Carpet keeps. 

In [ ]:
f.close() # make sure to close an hdf5 file when you're done with it

Let's read in the lapse and the grid data. We can read in all the relevant files using the Python glob tool.

In [ ]:
from glob import glob

grid_filenames = sorted(glob('qc0-mclachlan-cell-t0/grid-coordinates.file_*.h5'))
lapse_filenames = sorted(glob('qc0-mclachlan-cell-t0/admbase-lapse.file_*.h5'))

grid_filenames

And now we can read in all the relevant components:

In [ ]:
grid = {}
for filename in grid_filenames:
    with h5py.File(filename,'r') as f:
        for k,v in f.items():
            try: # this little trick reads only data sets from the root group
                grid[k] = v.value
            except:
                pass
lapse = {}
for filename in lapse_filenames:
    with h5py.File(filename,'r') as f:
        for k,v in f.items():
            try: # this little trick reads only data sets from the root group
                lapse[k] = v.value
            except:
                pass

In [ ]:
list(grid.keys())

In [ ]:
list(lapse.keys())

A commom trick I like to use is to define functions to access the various field names. For example:

In [ ]:
def get_coord(d, rl, c, it = 0, tl = 0):
    return grid['GRID::{} it={} tl={} rl={} c={}'.format(d,it,tl,rl,c)]
def get_lapse(rl,c,it=0,tl=0):
    return lapse['ADMBASE::alp it={} tl={} rl={} c={}'.format(it,tl,rl,c)]

Now we can load in and plot a component. (Note that the index ordering is fortran order. The indices go $zyx$.) For example:

In [ ]:
rl = 6
c = 3
alpha = get_lapse(rl,c)
X,Y,Z = [get_coord(d,rl,c) for d in ['x','y','z']]
iz = int(Z.shape[2]/2)
mesh = plt.pcolormesh(X[iz],Y[iz],alpha[iz],rasterized=True)
cbar = plt.colorbar()
cbar.set_label(r'$\alpha$')
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.savefig('../figures/lapse2d.pdf',bbox_inches='tight',dpi=600)

## Exercise

This simulation data contains two black holes in-spiraling in puncture coordinates. Can you find their coordinate locations? 

**HINT:** The lapse $\alpha$ is minimal near the punctures.